In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
import os


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
eat_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/광진구_식당_detail.csv")
eat_review = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/광진구_식당_review.csv")

In [4]:
eat_info

,Unnamed: 0,가게이름,가게 주소,음식 종류,가격 범위,리뷰개수,평점,lat,lng
0,1,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,만원 미만,72,4.5,37.554019,127.094133
1,2,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,만원 미만,94,4.2,37.545355,127.085301
2,3,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,만원 미만,40,4.0,37.546406,127.075574
3,4,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,만원 미만,13,4.0,37.564606,127.081460
4,5,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,만원 미만,9,4.0,37.541762,127.065878
...,...,...,...,...,...,...,...,...,...
728,729,화양주막,서울특별시 광진구 아차산로27길 4 1F,일반 주점,2만원-3만원,1,NaN,37.541787,127.067684
729,730,마우로아,서울특별시 광진구 능동로 320 1F,칵테일 / 와인,3만원-4만원,10,4.2,37.558149,127.080287
730,731,글래시,서울특별시 광진구 능동로 23-14 1F,칵테일 / 와인,3만원-4만원,19,3.5,37.533938,127.066940
731,732,리바,서울특별시 광진구 워커힐로 177 비스타 워커힐 1F,칵테일 / 와인,4만원 이상,4,NaN,37.554370,127.111210


In [5]:
eat_review

,Unnamed: 0,가게이름,리뷰아이디,리뷰,맛평가
0,1,맛이차이나,김성우,상수 [맛이차이나]<전국구 팔보채와 아쉬운 그 친구들..>⠀다시 방문한 상수의 [맛...,괜찮다
1,2,원조할아버지손두부,달려요달려,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,맛있다
2,3,원조할아버지손두부,ultraus,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,맛있다
3,4,원조할아버지손두부,세상의 모든 음식을 정복하겠어,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,맛있다
4,5,원조할아버지손두부,조선의장희빈,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,맛있다
...,...,...,...,...,...
7141,7142,맛이차이나,조냠냠,"진짜 맛이 차이나는지는 모르겠고 ㅎ깐풍기 소짜에 크림새우 세트 하나에게살스프, 소고...",별로
7142,7143,맛이차이나,굿걸,전체적으로 맛이 괜찮았지만왜인지 다시 가고 싶지는 않은 느낌?몇 년만에 다시 갔더니...,괜찮다
7143,7144,맛이차이나,민주,기대를 너무 하고 가지 않길 탕수육은 초마가 훨씬 더 맛있었고 칠리새우는 다 먹지도 않음,별로
7144,7145,맛이차이나,떠뽀끼쥬떼연,어향가지가 맛있는 중국집 런치 세트가 가성비 갑인 중국집 사실 간짜장이 먹고싶어서 ...,괜찮다


In [6]:
eat_info.columns

Index(['Unnamed: 0', '가게이름', '가게 주소', '음식 종류', '가격 범위', '리뷰개수', '평점', 'lat',
       'lng'],
      dtype='object')

In [7]:
eat_info.rename(columns={'가게이름': 'name'}, inplace =True)
eat_info.rename(columns={'리뷰': 'review'}, inplace =True)
eat_info.rename(columns={'평점': 'rating'}, inplace =True)
eat_info.rename(columns={'리뷰개수': 'rvw_cnt'}, inplace =True)
eat_info.rename(columns={'가게 주소': 'address'}, inplace =True)
eat_info.rename(columns={'음식 종류': 'category'}, inplace =True)
eat_info.rename(columns={'가격 범위': 'price'}, inplace =True)




eat_review.rename(columns={'가게이름': 'res_name'}, inplace =True)
eat_review.rename(columns={'리뷰아이디':  'user_name'}, inplace =True)
eat_review.rename(columns={'리뷰':  'review'}, inplace =True)
eat_review.rename(columns={'맛평가':  'rating'}, inplace =True)

eat_info.drop(['Unnamed: 0'],axis=1, inplace=True)
eat_review.drop(['Unnamed: 0'],axis=1, inplace=True)



In [8]:
eat_info.columns

Index(['name', 'address', 'category', 'price', 'rvw_cnt', 'rating', 'lat',
       'lng'],
      dtype='object')

In [9]:
eat_info = eat_info[eat_info['name'].notnull()]
eat_review = eat_review[eat_review['user_name'].notnull()]
eat_info.reset_index(drop=True,inplace=True)
eat_review.reset_index(drop=True,inplace=True)

In [10]:
#delete overlapped data
for id in eat_info.index:
    if list(eat_info['name']).count(eat_info.loc[id,'name'])>1:
        eat_info.drop(id,inplace=True)

In [11]:
#label encoding, Place_name -> p_id, User_name -> u_id
le = LabelEncoder()
le.fit(eat_info['name'])
eat_info['p_id'] = le.transform(eat_info['name'])
eat_review['p_id'] = le.transform(eat_review['res_name'])

user_le = LabelEncoder()
user_le.fit(eat_review['user_name'])
eat_review['u_id'] = user_le.transform(eat_review['user_name'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [12]:
# delete punctuation marks
def delete_punctuation_marks_review(review):
    review = review.replace('.', '. ');
    review = review.replace('!', '! ');
    review = review.replace('(', '( ');
    review = review.replace(')', ') ');
    review = review.replace('^', '');
    review = review.replace('*', '');
    review = review.replace('-', ' ');
    review = review.replace('\n', '. ');
    review = review.replace('\"', '. ');
    return review

In [13]:
eat_info.head()


,name,address,category,price,rvw_cnt,rating,lat,lng,p_id
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,만원 미만,72,4.5,37.554019,127.094133,448
1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,만원 미만,94,4.2,37.545355,127.085301,309
2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,만원 미만,40,4.0,37.546406,127.075574,63
3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,만원 미만,13,4.0,37.564606,127.081460,568
4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,만원 미만,9,4.0,37.541762,127.065878,34


In [14]:
eat_review.head()


,res_name,user_name,review,rating,p_id,u_id
0,맛이차이나,김성우,상수 [맛이차이나]<전국구 팔보채와 아쉬운 그 친구들..>⠀다시 방문한 상수의 [맛...,괜찮다,191,876
1,원조할아버지손두부,달려요달려,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,맛있다,448,1049
2,원조할아버지손두부,ultraus,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,맛있다,448,674
3,원조할아버지손두부,세상의 모든 음식을 정복하겠어,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,맛있다,448,1596
4,원조할아버지손두부,조선의장희빈,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,맛있다,448,2150


In [15]:
eat_info['reviews'] = ''


In [16]:
# from requests import Request
# from requests import Session


# def KOtoEN(review):
#     s=Session()

#     headers ={
#         "X-Naver-Client-Id": "NsHid2ZsKcP5QclyL66L",
#         "X-Naver-Client-Secret": "OPb4L85YOo",
#     }

#     payload ={
#         'source':'ko',
#         'target':'en',
#         'text': review,
#     }

#     url ="https://openapi.naver.com/v1/papago/n2mt"
#     # "https://openapi.naver.com/v1/language/translate"

#     req =Request('POST', url, data=payload, headers=headers)
#     prepped =req.prepare()
#     res =s.send(prepped)
#     print(res.json())
#     # result =res.json()['message']['result']['translatedText']
#     return result

In [17]:
review_1=eat_review['review'][1]
review_1

'예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 있는데 이집은 전혀 그런거 없이 고소하다.'

In [18]:
# KOtoEN(review_1)

In [19]:
# ts = Translator()
# eat_review['review_eng'] = ''

# #translating review
# for id in tqdm(eat_review.index):
#     review = eat_review.loc[id,'review']
    
#     if type(review)==float:
#         continue
        
#     review_pre = delete_punctuation_marks_review(review)
    
#     try:
#         review_ts = KOtoEN(review_pre)
#     except:
#         review_ts = ''

#     print(review_ts)
    
#     eat_review.loc[id,'review_eng'] = review_ts

In [20]:
# ts = Translator()

# ts.translate(review_1, source='ko', target='en', verbose=False)

In [21]:
# ts = Translator()
# eat_review['review_eng'] = ''

# #translating review
# for id in tqdm(eat_review.index):
#     review = eat_review.loc[id,'review']
    
#     if type(review)==float:
#         continue
        
#     review_pre = delete_punctuation_marks_review(review)
    
#     try:
#         review_ts = ts.translate(review_pre, source='ko', target='en', verbose=False)
#     except:
#         review_ts = ''
    
#     eat_review.loc[id,'review_eng'] = review_ts

In [22]:
for id in eat_info['p_id']:
    review_all = ''
    
    for review in eat_review[eat_review['p_id']==id]['review']:
        review_all+=review +'. '
        
    eat_info.loc[eat_info['p_id']==id,'reviews'] = review_all

In [23]:
eat_info.head()


,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,만원 미만,72,4.5,37.554019,127.094133,448,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...
1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,만원 미만,94,4.2,37.545355,127.085301,309,서울특별시 광진구 구의2동에 위치한 서북면옥에서 물냉면(9천원)과 접시만두 6조각(...
2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,만원 미만,40,4.0,37.546406,127.075574,63,기본에 충실해서 더 맛있었던 핫도그였습니다. 들어가서 음식이 나오기까지는 꽤 시간이...
3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,만원 미만,13,4.0,37.564606,127.081460,568,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다..1위가...
4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,만원 미만,9,4.0,37.541762,127.065878,34,혼밥하기 좋은곳.김밥도 괜찮고 잔치국수도 깔끔하니 괜찮다.. 잔치국수와 건대김밥 주...


In [24]:
eat_info['price'].unique()

array(['만원 미만', '만원-2만원', '2만원-3만원', '3만원-4만원', '4만원 이상'], dtype=object)

In [25]:
eat_info['price'].replace('만원 미만',5000, inplace=True)
eat_info['price'].replace('만원-2만원',15000, inplace=True)
eat_info['price'].replace('2만원-3만원',25000, inplace=True)
eat_info['price'].replace('3만원-4만원',35000, inplace=True)
eat_info['price'].replace('4만원 이상',50000, inplace=True)

In [26]:
eat_review.head()


,res_name,user_name,review,rating,p_id,u_id
0,맛이차이나,김성우,상수 [맛이차이나]<전국구 팔보채와 아쉬운 그 친구들..>⠀다시 방문한 상수의 [맛...,괜찮다,191,876
1,원조할아버지손두부,달려요달려,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,맛있다,448,1049
2,원조할아버지손두부,ultraus,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,맛있다,448,674
3,원조할아버지손두부,세상의 모든 음식을 정복하겠어,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,맛있다,448,1596
4,원조할아버지손두부,조선의장희빈,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,맛있다,448,2150


In [27]:
eat_review['rating'].unique()

array(['괜찮다', '맛있다', '별로'], dtype=object)

In [28]:
eat_review['rating'].replace('맛있다',5, inplace=True)
eat_review['rating'].replace('괜찮다',3, inplace=True)
eat_review['rating'].replace('별로',0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [29]:
eat_info

,name,address,category,price,rvw_cnt,rating,lat,lng,p_id,reviews
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,5000,72,4.5,37.554019,127.094133,448,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...
1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,5000,94,4.2,37.545355,127.085301,309,서울특별시 광진구 구의2동에 위치한 서북면옥에서 물냉면(9천원)과 접시만두 6조각(...
2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,5000,40,4.0,37.546406,127.075574,63,기본에 충실해서 더 맛있었던 핫도그였습니다. 들어가서 음식이 나오기까지는 꽤 시간이...
3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,5000,13,4.0,37.564606,127.081460,568,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다..1위가...
4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,5000,9,4.0,37.541762,127.065878,34,혼밥하기 좋은곳.김밥도 괜찮고 잔치국수도 깔끔하니 괜찮다.. 잔치국수와 건대김밥 주...
...,...,...,...,...,...,...,...,...,...,...
728,화양주막,서울특별시 광진구 아차산로27길 4 1F,일반 주점,25000,1,NaN,37.541787,127.067684,663,곱도리탕.묵사발.미숫가루막걸리 맛있는곳ㅎㅎ곱창이 4줄이나 있어서 29000원가격이 ...
729,마우로아,서울특별시 광진구 능동로 320 1F,칵테일 / 와인,35000,10,4.2,37.558149,127.080287,180,성수 유명한 맛집에서 이제는 군자 맛집으로너무 가보고싶었던 곳이라 엄마랑 동생이랑 ...
730,글래시,서울특별시 광진구 능동로 23-14 1F,칵테일 / 와인,35000,19,3.5,37.533938,127.066940,64,뚝섬역에서 가까운 글래시 와인바에요 ! 와인바 찾던 중 우연히 발견했는데 가격도 맛...
731,리바,서울특별시 광진구 워커힐로 177 비스타 워커힐 1F,칵테일 / 와인,50000,4,NaN,37.554370,127.111210,173,제주도에서 하는 빛의 벙커를 서울에서도 한다고 하여워커힐 방문!빛의 시어터 다 보고...


In [30]:
eat_review

,res_name,user_name,review,rating,p_id,u_id
0,맛이차이나,김성우,상수 [맛이차이나]<전국구 팔보채와 아쉬운 그 친구들..>⠀다시 방문한 상수의 [맛...,3,191,876
1,원조할아버지손두부,달려요달려,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,5,448,1049
2,원조할아버지손두부,ultraus,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,5,448,674
3,원조할아버지손두부,세상의 모든 음식을 정복하겠어,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,5,448,1596
4,원조할아버지손두부,조선의장희빈,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,5,448,2150
...,...,...,...,...,...,...
7125,맛이차이나,조냠냠,"진짜 맛이 차이나는지는 모르겠고 ㅎ깐풍기 소짜에 크림새우 세트 하나에게살스프, 소고...",0,191,2146
7126,맛이차이나,굿걸,전체적으로 맛이 괜찮았지만왜인지 다시 가고 싶지는 않은 느낌?몇 년만에 다시 갔더니...,3,191,815
7127,맛이차이나,민주,기대를 너무 하고 가지 않길 탕수육은 초마가 훨씬 더 맛있었고 칠리새우는 다 먹지도 않음,0,191,1402
7128,맛이차이나,떠뽀끼쥬떼연,어향가지가 맛있는 중국집 런치 세트가 가성비 갑인 중국집 사실 간짜장이 먹고싶어서 ...,3,191,1119


In [31]:
eat_info.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restaurant_info.csv', index=False)
eat_review.to_csv('/content/drive/MyDrive/Colab Notebooks/data/광진구_restaurant_review.csv', index=False)